<a href="https://colab.research.google.com/github/dlmacedo/starter-academic/blob/master/example_fine_tuning_hf_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning a deep-learning model with HuggingFace `🤗Datasets` library and PyTorch

Based on its [Quick tour example](https://huggingface.co/docs/datasets/quicktour.html)

Colab author: Manuel Romero / [@mrm8488](https://twitter.com/mrm8488)

In [ ]:
!nvidia-smi

Fri Sep 11 14:57:04 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q datasets
!pip install -q transformers

In [ ]:
# Make sure that we have a recent version of pyarrow in the session before we continue - otherwise reboot Colab to activate it
import pyarrow
if int(pyarrow.__version__.split('.')[1]) < 16 and int(pyarrow.__version__.split('.')[0]) == 0:
    import os
    os.kill(os.getpid(), 9)

### Load the model and the tokenizer

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

### Load and process the dataset

In [ ]:
from datasets import load_dataset
dataset = load_dataset('glue', 'mrpc', split='train')

PyTorch version 1.6.0+cu101 available.
TensorFlow version 2.3.0 available.
https://raw.githubusercontent.com/huggingface/datasets/1.0.1/datasets/glue/glue.py not found in cache or force_download set to True, downloading to /root/.cache/huggingface/datasets/tmpzqmcrfui


storing https://raw.githubusercontent.com/huggingface/datasets/1.0.1/datasets/glue/glue.py in cache at /root/.cache/huggingface/datasets/8a575b8341116e1cf0f3a928d70f7ee7ec1aee5e6620744d30d5331a2be68979.d804a9b67563ab7de5bb068d5eccc0eff8cf0849041ad2e8afff1beb8a14544d.py
creating metadata file for /root/.cache/huggingface/datasets/8a575b8341116e1cf0f3a928d70f7ee7ec1aee5e6620744d30d5331a2be68979.d804a9b67563ab7de5bb068d5eccc0eff8cf0849041ad2e8afff1beb8a14544d.py


https://raw.githubusercontent.com/huggingface/datasets/1.0.1/datasets/glue/dataset_infos.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/datasets/tmpa7esm82v


storing https://raw.githubusercontent.com/huggingface/datasets/1.0.1/datasets/glue/dataset_infos.json in cache at /root/.cache/huggingface/datasets/e9129383f6197a6e76ba55cb5a1dfc2cc28e085b4585b48a3cf8978344805837.03e16a366649d9f5c63e615ccbc58466c013cc8677c5be1b52636c46e597c13c
creating metadata file for /root/.cache/huggingface/datasets/e9129383f6197a6e76ba55cb5a1dfc2cc28e085b4585b48a3cf8978344805837.03e16a366649d9f5c63e615ccbc58466c013cc8677c5be1b52636c46e597c13c
Checking /root/.cache/huggingface/datasets/8a575b8341116e1cf0f3a928d70f7ee7ec1aee5e6620744d30d5331a2be68979.d804a9b67563ab7de5bb068d5eccc0eff8cf0849041ad2e8afff1beb8a14544d.py for additional imports.
Creating main folder for dataset https://raw.githubusercontent.com/huggingface/datasets/1.0.1/datasets/glue/glue.py at /root/.cache/huggingface/modules/datasets_modules/datasets/glue
Creating specific version folder for dataset https://raw.githubusercontent.com/huggingface/datasets/1.0.1/datasets/glue/glue.py at /root/.cache/hugg

Dataset not on Hf google storage. Downloading and preparing it from source
  0%|          | 0/3 [00:00<?, ?it/s]Couldn't get ETag version for url https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2Fmrpc_dev_ids.tsv?alt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc
https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2Fmrpc_dev_ids.tsv?alt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc not found in cache or force_download set to True, downloading to /root/.cache/huggingface/datasets/downloads/tmpp_lhfbdh


...

storing https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_test.txt in cache at /root/.cache/huggingface/datasets/downloads/d0f75e90c732a9847ec38471fddece4ebcaad09dd1958467e2b00c6a3cbd31a9
creating metadata file for /root/.cache/huggingface/datasets/downloads/d0f75e90c732a9847ec38471fddece4ebcaad09dd1958467e2b00c6a3cbd31a9
100%|██████████| 3/3 [00:06<00:00,  2.14s/it]
Checksum Computation took 0.0 min
All the checksums matched successfully for dataset source files
Generating split train


Done writing 3668 examples in 943851 bytes /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/930e9d141872db65102cabb9fa8ac01c11ffc8a1b72c2e364d8cdda4610df542.incomplete/glue-train.arrow.
Generating split validation


Done writing 408 examples in 105887 bytes /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/930e9d141872db65102cabb9fa8ac01c11ffc8a1b72c2e364d8cdda4610df542.incomplete/glue-validation.arrow.
Generating split test


Done writing 1725 examples in 442418 bytes /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/930e9d141872db65102cabb9fa8ac01c11ffc8a1b72c2e364d8cdda4610df542.incomplete/glue-test.arrow.
All the splits matched successfully.
Constructing Dataset for split train, from /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/930e9d141872db65102cabb9fa8ac01c11ffc8a1b72c2e364d8cdda4610df542


Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/930e9d141872db65102cabb9fa8ac01c11ffc8a1b72c2e364d8cdda4610df542. Subsequent calls will reuse this data.


In [ ]:
len(dataset)

3668

In [ ]:
dataset[0]

{'idx': 0,
 'label': 1,
 'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'}

In [ ]:
dataset.features

{'idx': Value(dtype='int32', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None),
 'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None)}

In [ ]:
# dataset.filter(lambda example: example['label'] == dataset.features['label'].str2int('equivalent'))[0]

In [ ]:
# dataset.filter(lambda example: example['label'] == dataset.features['label'].str2int('not_equivalent'))[0]

#### Tokenizing the dataset

In [ ]:
def encode(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding='max_length')
dataset = dataset.map(encode, batched=True)
dataset[0]

#### Formatting the dataset

In [ ]:
dataset = dataset.map(lambda examples: {'labels': examples['label']}, batched=True)

Testing the mapped function outputs
Testing finished, running the mapping function on the dataset
Caching processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/930e9d141872db65102cabb9fa8ac01c11ffc8a1b72c2e364d8cdda4610df542/cache-29a353f99e973486.arrow


Done writing 3668 examples in 46089667 bytes /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/930e9d141872db65102cabb9fa8ac01c11ffc8a1b72c2e364d8cdda4610df542/tmpzgbty91d.


In [ ]:
import torch
dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=8)
next(iter(dataloader))

Set __getitem__(key) output type to torch for ['input_ids', 'token_type_ids', 'attention_mask', 'labels'] columns  (when key is int or slice) and don't output other (un-formatted) columns.
/usr/local/lib/python3.6/dist-packages/datasets/arrow_dataset.py:835: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)


{'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'input_ids': tensor([[  101,  7277,  2180,  ...,     0,     0,     0],
         [  101, 10684,  2599,  ...,     0,     0,     0],
         [  101,  1220,  1125,  ...,     0,     0,     0],
         ...,
         [  101, 16944,  1107,  ...,     0,     0,     0],
         [  101,  1109, 11896,  ...,     0,     0,     0],
         [  101,  1109,  4173,  ...,     0,     0,     0]]),
 'labels': tensor([1, 0, 1, 0, 1, 1, 0, 1]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]])}

#### Training

In [ ]:
from tqdm import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.train().to(device)
optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)

In [ ]:
for epoch in range(3):
    for i, batch in enumerate(tqdm(dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if i % 10 == 0:
            print(f"loss: {loss}")

  0%|          | 1/459 [00:00<03:48,  2.01it/s]

loss: 0.9393188953399658


  2%|▏         | 11/459 [00:03<02:08,  3.49it/s]

loss: 0.773149847984314


  5%|▍         | 21/459 [00:06<02:04,  3.50it/s]

...

 96%|█████████▌| 441/459 [02:03<00:05,  3.57it/s]

loss: 0.10758677870035172


 98%|█████████▊| 451/459 [02:06<00:02,  3.55it/s]

loss: 0.19609297811985016


100%|██████████| 459/459 [02:08<00:00,  3.57it/s]
